In [5]:
import pandas as pd
from dash import dcc, html
import dash_bootstrap_components as dbc
import datetime as dt
import settings as st
from dotenv import load_dotenv
import os 

#df = pd.read_csv('Clear_dash_new.csv')

# Загрузка переменных из .env файла
load_dotenv()

# Получение путей из переменных окружения
excel_file_path = os.getenv('EXCEL_FILE_PATH')
sheet_name = os.getenv('SHEET_NAME')  # значение по умолчанию

try:
    df = pd.read_excel(
        excel_file_path,
        skiprows=1,
        sheet_name=sheet_name,
        engine='openpyxl'  # для .xlsx файлов
    )
    
    print("Данные успешно загружены:")
    print(df.head())
    
except Exception as e:
    print(f"Ошибка при чтении файла: {e}")



df = df.rename(columns={'Дата счёта':'Acc_date', '№ дог':'Contr_num', 
                        'Заказчик':'Customer', 'Подразделение': 'Department',
                        'Наименование работ': 'Job_desc', 'Сумма': 'Sum',
                        'Дата оплаты': 'Payment_date', 'Дата акта': 'Exec_date',
                        'Ответственный':'Responsible', 'Срок платежа':'Payment_time',
                        'Cрок выполнения':'Exec_time'})

df.drop(['Contr_num', 'Job_desc'], axis=1, inplace=True)

Данные успешно загружены:
   Номер счёта Дата счёта       № дог         Заказчик    Исполнитель  \
0          1.0 2025-01-09         NaN  Стоматология 20  ИП Литвиненко   
1          2.0 2025-01-09         NaN     Призма-центр  ИП Литвиненко   
2          3.0 2025-01-10  № 25/12-24     Призма-центр  ИП Литвиненко   
3          1.0 2025-01-10         NaN     Призма-центр    ИП Маклаков   
4          1.0 2025-01-13         NaN           Сервер        Редберг   

  Подразделение                                 Наименование работ      Сумма  \
0           ВИК                                     Диагностика ВУ    3900.00   
1         ОЭЗиС                       Ремонт части помещения № 243   11000.00   
2         ОЭЗиС  Косметический ремонт лестничной клетки между 4...  133310.08   
3         ОЭЗиС                                         Ремонт ГВС   25178.40   
4         ОЭЗиС             Ремонт трубопровода подпитки котельной   12160.00   

  Дата оплаты  Дата акта Ответственный  Срок пла

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Номер счёта   172 non-null    float64       
 1   Acc_date      176 non-null    datetime64[ns]
 2   Customer      176 non-null    object        
 3   Исполнитель   176 non-null    object        
 4   Department    176 non-null    object        
 5   Sum           181 non-null    float64       
 6   Payment_date  164 non-null    datetime64[ns]
 7   Exec_date     161 non-null    datetime64[ns]
 8   Responsible   176 non-null    object        
 9   Payment_time  164 non-null    float64       
 10  Exec_time     161 non-null    float64       
dtypes: datetime64[ns](3), float64(4), object(4)
memory usage: 19.1+ KB


In [7]:
df = df.dropna(subset=['Acc_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 176 entries, 0 to 175
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Номер счёта   172 non-null    float64       
 1   Acc_date      176 non-null    datetime64[ns]
 2   Customer      176 non-null    object        
 3   Исполнитель   176 non-null    object        
 4   Department    176 non-null    object        
 5   Sum           175 non-null    float64       
 6   Payment_date  164 non-null    datetime64[ns]
 7   Exec_date     161 non-null    datetime64[ns]
 8   Responsible   176 non-null    object        
 9   Payment_time  164 non-null    float64       
 10  Exec_time     161 non-null    float64       
dtypes: datetime64[ns](3), float64(4), object(4)
memory usage: 16.5+ KB


In [8]:
df['month'] = df['Acc_date'].dt.strftime('%Y-%m')  # Формат "2024-01"
df.head()

,Номер счёта,Acc_date,Customer,Исполнитель,Department,Sum,Payment_date,Exec_date,Responsible,Payment_time,Exec_time,month
0,1.0,2025-01-09,Стоматология 20,ИП Литвиненко,ВИК,3900.00,2025-01-09,2025-01-31,Ярчев,0.0,22.0,2025-01
1,2.0,2025-01-09,Призма-центр,ИП Литвиненко,ОЭЗиС,11000.00,2025-01-14,2025-01-09,Бурик,5.0,0.0,2025-01
2,3.0,2025-01-10,Призма-центр,ИП Литвиненко,ОЭЗиС,133310.08,2025-01-14,2025-01-10,Бурик,4.0,0.0,2025-01
3,1.0,2025-01-10,Призма-центр,ИП Маклаков,ОЭЗиС,25178.40,2025-01-14,2025-01-10,Бурик,4.0,0.0,2025-01
4,1.0,2025-01-13,Сервер,Редберг,ОЭЗиС,12160.00,2025-01-14,2025-01-13,Бурик,1.0,0.0,2025-01
